In [1]:
%load_ext autoreload
%autoreload 2
%run main.py
from main import Crop

In [2]:
product_iri = ProductIRI("http://data.europa.eu/xsp/cn2024/100500000080")
amount = 7
unit = ProductIRI("https://vocab.sentier.dev/units/unit/KiloGM")

In [3]:
product_iri

<http://data.europa.eu/xsp/cn2024/100500000080>: 1005 Maize (corn) (product)

In [4]:
user_input = UserInput(
    product_iri=product_iri,
    unit = unit,
    amount=amount
    #climate_type ='wet'
)

In [5]:
user_input

UserInput(product_iri=<http://data.europa.eu/xsp/cn2024/100500000080>: 1005 Maize (corn) (product), unit=<https://vocab.sentier.dev/units/unit/KiloGM>: Missing label (product), amount=7.0, crop_yield=None, fertilizer_amount=None, climate_type=None, spatial_context=GeonamesIRI('https://sws.geonames.org/6295630/'), begin_date=None, end_date=None)

In [6]:
run_config = RunConfig()

In [7]:
ProductIRI(
            "http://data.europa.eu/xsp/cn2024/060011000090"
            )

<http://data.europa.eu/xsp/cn2024/060011000090>: SECTION II - VEGETABLE PRODUCTS (product)

In [8]:
crop = Crop(user_input = user_input, run_config = run_config)

import create_data
create_data.create_example_local_datastorage()

x = crop.get_model_data(product=ProductIRI(
            "http://data.europa.eu/xsp/cn2024/060011000090"
            ), kind=DatasetKind.BOM )

x['exactMatch'][0].dataframe

agridata = crop.get_master_db()

In [10]:
crop.run()

INFO: create data
INFO: Getting climate
INFO: Getting crop yield and fertilizer amount


18:41:50 [info     ] Retrieved 0 triples from https://fuseki.d-d-s.ch/skosmos/query
18:41:50 [info     ] Retrieved 865 triples from https://fuseki.d-d-s.ch/skosmos/query
18:41:50 [info     ] Retrieved 0 triples from https://fuseki.d-d-s.ch/skosmos/query
18:41:50 [info     ] Retrieved 865 triples from https://fuseki.d-d-s.ch/skosmos/query
18:41:50 [info     ] Retrieved 17 triples from https://fuseki.d-d-s.ch/skosmos/query


INFO: Found broader match for <http://data.europa.eu/xsp/cn2024/060011000090>: SECTION II - VEGETABLE PRODUCTS (product)
INFO:                                         crop_iri  \
3  http://data.europa.eu/xsp/cn2024/060011000090   
4  http://data.europa.eu/xsp/cn2024/060011000090   
5  http://data.europa.eu/xsp/cn2024/060011000090   

                                        fert_iri climate_type fert_type  \
3  http://data.europa.eu/xsp/cn2024/310200000080      default   default   
4  http://data.europa.eu/xsp/cn2024/310200000080      default   default   
5  http://data.europa.eu/xsp/cn2024/310200000080      default   default   

  value_type  emission_factor  emission [kg_N20/ha]  
3      value            0.010                  1.10  
4        min            0.001                  0.11  
5        max            0.018                  1.98  
INFO: Getting emission from fertilizer


In [43]:
crop.crop_yield_data

AttributeError: 'Crop' object has no attribute 'crop_yield_data'

In [31]:
from function import format_df

for i in agridata["exactMatch"]:
    display(i.columns)

[{'iri': 'http://purl.org/dc/terms/date',
  'unit': 'https://www.w3.org/2001/XMLSchema#string'},
 {'iri': 'http://purl.org/dc/terms/Location',
  'unit': 'https://www.w3.org/2001/XMLSchema#string'},
 {'iri': 'http://data.europa.eu/xsp/cn2024/060011000090',
  'unit': 'https://www.w3.org/2001/XMLSchema#string'},
 {'iri': 'http://data.europa.eu/xsp/cn2024/310200000080',
  'unit': 'https://vocab.sentier.dev/units/unit/KiloGM-PER-M2'}]

In [9]:
resulting_demands = crop.run()

INFO: Getting master db
INFO: Getting crop yield and fertilizer amount


13:41:33 [info     ] Retrieved 17 triples from https://fuseki.d-d-s.ch/skosmos/query


INFO: Found broader match for <http://data.europa.eu/xsp/cn2024/060011000090>: SECTION II - VEGETABLE PRODUCTS (product)
INFO:                                         crop_iri  \
3  http://data.europa.eu/xsp/cn2024/060011000090   
4  http://data.europa.eu/xsp/cn2024/060011000090   
5  http://data.europa.eu/xsp/cn2024/060011000090   

                                        fert_iri climate_type fert_type  \
3  http://data.europa.eu/xsp/cn2024/310200000080      default   default   
4  http://data.europa.eu/xsp/cn2024/310200000080      default   default   
5  http://data.europa.eu/xsp/cn2024/310200000080      default   default   

  value_type  emission_factor  emission [kg_N20/ha]  
3      value            0.010                  1.10  
4        min            0.001                  0.11  
5        max            0.018                  1.98  
INFO: Getting emission from fertilizer


In [ ]:
resulting_demands

# Main tst SentierModel

In [ ]:
import sentier_data_tools

In [ ]:
sentier_data_tools.__version__

In [ ]:
%run main_tst.py

# n2OToAirInorganicFertiliserDirect_dev.py

In [1]:
import n2OToAirInorganicFertiliserDirect_dev as n2o

In [2]:
import pandas as pd

In [9]:
product_iri = ProductIRI("http://data.europa.eu/xsp/cn2024/100500000080")

In [10]:
run(product_iri, 7)

Exception: File `'(product_iri,'` not found.

# function.py

In [5]:
from sentier_data_tools.iri import ProductIRI

In [6]:
import function

In [7]:
product_iri = ProductIRI("http://data.europa.eu/xsp/cn2024/100500000080")

In [4]:
product_iri.display()

'<http://data.europa.eu/xsp/cn2024/100500000080>: 1005 Maize (corn) (product)'

In [5]:
unique_product = [ product_iri.broader()[0], product_iri.broader()[1], product_iri.narrower()[0]]

09:52:08 [info     ] Retrieved 17 triples from https://fuseki.d-d-s.ch/skosmos/query
09:52:08 [info     ] Retrieved 17 triples from https://fuseki.d-d-s.ch/skosmos/query
09:52:08 [info     ] Retrieved 7 triples from https://fuseki.d-d-s.ch/skosmos/query


In [6]:
unique_product

[<http://data.europa.eu/xsp/cn2024/100021000090>: CHAPTER 10 - CEREALS (product),
 <http://data.europa.eu/xsp/cn2024/060011000090>: SECTION II - VEGETABLE PRODUCTS (product),
 <http://data.europa.eu/xsp/cn2024/100590000080>: Other (product)]

In [7]:
match_IRI = function.find_match_IRI(product_iri, unique_product)

09:52:09 [info     ] Retrieved 17 triples from https://fuseki.d-d-s.ch/skosmos/query


INFO: Found broader match for <http://data.europa.eu/xsp/cn2024/100021000090>: CHAPTER 10 - CEREALS (product)


In [16]:
df = pd.read_csv("../docs/EF.csv", sep=';')
productIRI_columns_list = ['crop_iri', 'fert_iri']

df[productIRI_columns_list] = df[productIRI_columns_list].map(lambda x : ProductIRI(x))

In [18]:
df['crop_iri'][0]

<http://data.europa.eu/xsp/cn2024/060011000090>: SECTION II - VEGETABLE PRODUCTS (product)

In [19]:
df_emission_factors['crop_iri'][0]

'http://data.europa.eu/xsp/cn2024/060011000090'